<a href="https://colab.research.google.com/github/TomJiX/ML_start/blob/main/online_cv_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Package

In [ ]:
!pip install face_recognition

In [12]:
#Package
import face_recognition
import cv2    
import math
import time
import numpy as np
import sys
import seaborn as sns
from datetime import datetime
from google.colab.patches import cv2_imshow
import os

#Data import

In [ ]:
from google.colab import files
files.upload()

#Is mouth open

In [5]:
# Mouth open/close
def get_lip_height(lip):
    sum=0
    for i in [2,3,4]:
        # distance between two near points up and down
        distance = math.sqrt( (lip[i][0] - lip[12-i][0])**2 +
                              (lip[i][1] - lip[12-i][1])**2   )
        sum += distance
    return sum / 3

def get_mouth_height(top_lip, bottom_lip):
    sum=0
    for i in [8,9,10]:
        # distance between two near points up and down
        distance = math.sqrt( (top_lip[i][0] - bottom_lip[18-i][0])**2 + 
                              (top_lip[i][1] - bottom_lip[18-i][1])**2   )
        sum += distance
    return sum / 3

def is_mouth_open(face_landmarks):
    top_lip = face_landmarks['top_lip']
    bottom_lip = face_landmarks['bottom_lip']

    top_lip_height = get_lip_height(top_lip)
    bottom_lip_height = get_lip_height(bottom_lip)
    mouth_height = get_mouth_height(top_lip, bottom_lip)

    # if mouth is open more than lip height * ratio, return true.
    ratio = 0.5
    print('top_lip_height: %.2f, bottom_lip_height: %.2f, mouth_height: %.2f, min*ratio: %.2f'
          % (top_lip_height,bottom_lip_height,mouth_height, min(top_lip_height, bottom_lip_height) * ratio))

    if mouth_height > min(top_lip_height, bottom_lip_height) * ratio:
        return True
    else:
        return False

def mouth_open_close(filename=""):
    # Get a reference to webcam #0 (the default one)
    if len(filename)==0:
      video_capture = cv2.VideoCapture(2)
    else :
      video_capture = cv2.VideoCapture(filename)
    video_capture.set(3,640)
    video_capture.set(4,480)
    # Load a sample picture and learn how to recognize it.

    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()
        frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        frame=cv2.flip(frame, 1)
        # Find all the faces and face enqcodings in the frame of video
        face_locations = face_recognition.face_locations(frame)
        #face_encodings = face_recognition.face_encodings(frame, face_locations)
        face_landmarks_list = face_recognition.face_landmarks(frame)

        # Loop through each face in this frame of video
        for (top, right, bottom, left), face_landmarks in zip(face_locations, face_landmarks_list):

            name="Unknown"
            
            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            
        
            
            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom), (right, bottom + 35), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom + 25), font, 1.0, (255, 255, 255), 1)


            # Display text for mouth open / close
            ret_mouth_open = is_mouth_open(face_landmarks)
            if ret_mouth_open is True:
                text = 'Mouth is open'
            else:
                text = 'Open your mouth'
            cv2.putText(frame, text, (left, top - 50), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)


        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

#Face bluring

In [80]:
# blur all faces
def face_blur(filename):
    video=[]
    video_capture = cv2.VideoCapture(filename)
    total_len = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    # Get a reference to webcam #0 (the default one)
    size=(int(video_capture.get(3)),int(video_capture.get(4)))
    i=1
    video_name=filename.split(".")[0]+".avi"
    print("\n"+video_name)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(video_name,fourcc, 24.0, size)

    while True:
        # Grab a single frame of video
        try:
          ret, frame = video_capture.read()
          #frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
          frame=cv2.flip(frame, 1)
          # Find all the faces and face enqcodings in the frame of video
          face_locations = face_recognition.face_locations(frame)
      
          # Loop through each face in this frame of video
          for location in face_locations:    
              top, right, bottom, left=location
              # Draw a box around the face
              cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
              sub_face = frame[top:bottom, left:right]
          
              # apply a gaussian blur on this new recangle image
              sub_face = cv2.GaussianBlur(sub_face,(23, 23), 30)
              # merge this blurry rectangle to our final image
              frame[top:top+sub_face.shape[0], left:left+sub_face.shape[1]] = sub_face
          out.write(frame)
          print("\r Data is being processed %s/%s"%(i,total_len),end="")
          i+=1
        except Exception as e:
          break
    out.release()

        

    

#Motion tracker

In [7]:
# Tracks selected objects
def motion_tracker_multiple(nb_of_obj,filename=""):
    # Set up tracker.
    # Instead of KCF, you can also use
    (major_ver,minor_ver,subminor_ver)=(cv2.__version__).split('.')
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[2]
    
    trackers=[None]*nb_of_obj
    for i in range(nb_of_obj):
        if int(minor_ver) < 3:
            trackers[i] = cv2.Tracker_create(tracker_type)
        else:
            if tracker_type == 'BOOSTING':
                trackers[i] = cv2.TrackerBoosting_create()
            if tracker_type == 'MIL':
                trackers[i] = cv2.TrackerMIL_create()
            if tracker_type == 'KCF':
                trackers[i] = cv2.TrackerKCF_create()
            if tracker_type == 'TLD':
                trackers[i] = cv2.TrackerTLD_create()
            if tracker_type == 'MEDIANFLOW':
                trackers[i] = cv2.TrackerMedianFlow_create()
            if tracker_type == 'GOTURN':
                trackers[i] = cv2.TrackerGOTURN_create()
            if tracker_type == 'MOSSE':
                trackers[i] = cv2.TrackerMOSSE_create()
            if tracker_type == "CSRT":
                trackers[i] = cv2.TrackerCSRT_create()
    
    # Read video
    if len(filename)==0:
      video_capture = cv2.VideoCapture(2)
    else :
      video_capture = cv2.VideoCapture(filename)
       #webcam (if not working try "0" or "1")
    video_capture.set(3,640)
    video_capture.set(4,480)
    

    # Read first frame.
    ok, frame = video_capture.read()
    #frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    frame=cv2.flip(frame, 1)
    if not ok:
        print('Cannot read video file')
        sys.exit()
        
    # Initialize tracker with first frame and bounding box
    ok_bboxs=[[None,None]]*nb_of_obj
    palette = 255*np.asarray(sns.color_palette(None, nb_of_obj))
    for i in range(nb_of_obj):
        bbox = cv2.selectROI(frame, False)
        ok = trackers[i].init(frame, bbox)
        ok_bboxs[i]=[ok,bbox]
    
    while True:
        # Read a new frame
        ok, frame = video_capture.read()
        frame=cv2.flip(frame, 1)
        if not ok:
            break
            
        # Start timer
        timer = cv2.getTickCount()
    
       
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        for i in range(nb_of_obj):
            # Update tracker
            ok_bboxs[i] = trackers[i].update(frame)
            # Draw bounding box
            if ok_bboxs[i][0]:
                # Tracking success
                p1 = (int(ok_bboxs[i][1][0]), int(ok_bboxs[i][1][1]))
                p2 = (int(ok_bboxs[i][1][0] + ok_bboxs[i][1][2]), int(ok_bboxs[i][1][1] + ok_bboxs[i][1][3]))
                cv2.rectangle(frame, p1, p2, palette[i], 2, 1)
                cv2.putText(frame, "object "+str(i), (p1[0] + 6, p2[1] + 25), cv2.FONT_HERSHEY_SIMPLEX, 1.0, palette[i], 1)

            else :
                # Tracking failure
                cv2.putText(frame, "Tracking failure detected for obj "+" "*i+str(i), (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
        
            # Display tracker type on frame
            cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
            
            # Display FPS on frame
            cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2)
            # Display result
            cv2.imshow("Tracking", frame)
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break

#Motion sensor

In [8]:
# Detects and tracks moving objects
def motion_sensor_v2(filename=""):

    ### input type ###
    if len(filename)==0:
      video_capture = cv2.VideoCapture(2)
    else :
      video_capture = cv2.VideoCapture(filename)
    
    #background substraction
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    fgbg = cv2.createBackgroundSubtractorKNN()
    
    firstFrame = None
    initBB2 = None
    fps = None
    differ = None
    now = ''
    framecounter = 0
    trackeron=0
    min_area=400

    tracked_items=[]

    while True:
        
        ret, frame = video_capture.read()
        thresh=frame
        # resize the frame to 500
        frame = cv2.resize(frame, (640, 480))
        frame=frame[240:,:]  # optionnal (hide non unseful zones to speed up process)
        
        #cv2.normalize(frame, frame, -100, 155, cv2.NORM_MINMAX)
        fgmask=frame
        framecounter = framecounter+1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # if the first frame is None, initialize it
        if firstFrame is None:
            firstFrame = gray
            continue
        
        fgmask = fgbg.apply(gray)
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
        
        # dilate the thresholded image to fill in holes, then find contours on thresholded image
        thresh = cv2.dilate(fgmask, None, iterations=3)
        
        if framecounter>10:
            cnts,hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

            # loop over the contours identified
            contourcount = 0
            for c in cnts:
                contourcount +=  1

            # if the contour is too small, ignore it
                (x, y, w, h) = cv2.boundingRect(c)
                if w*h<min_area:
                    continue
                initBB2 =(x,y,w,h)
        
                #for Trackers :   0:tracker, 1:box, 2:ok, 3:ok count, 4:static frame
            
                nb_of_items=len(tracked_items)
                dist=0

                if nb_of_items==0:
                    tracker=cv2.TrackerKCF_create()
                    tracked_items.append([tracker,initBB2,tracker.init(frame,initBB2),framecounter,0])    
                for track in tracked_items:
                    if distance(track[1][:2],[x,y],40):
                        dist+=1
                if dist==nb_of_items:
                    tracker=cv2.TrackerKCF_create()
                    tracked_items.append([tracker,initBB2,tracker.init(frame,initBB2),framecounter,0])
                    
            for track in tracked_items:
                previous_pos=track[1]
                track[2],track[1]=track[0].update(frame)
                if track[2]:
                    track[3]+=1
                if not distance(previous_pos[:2],track[1][:2],5):
                    track[4]+=1
                if framecounter-track[3]>30 or track[4]>=20:
                    tracked_items.remove(track)
            for track in tracked_items:
                if track[2]:
                    p1 = (int(track[1][0]), int(track[1][1]))
                    p2 = (int(track[1][0] + track[1][2]), int(track[1][1] + track[1][3]))
                    cv2.rectangle(frame, p1, p2, (0, 255, 0), 2, 1)

        # show the frame 
        cv2.imshow("Video stream", frame)

        cv2.imshow("Video stream2", thresh)
        
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key is pressed, break from the lop
        if key == ord("q"):
            break
        if key == ord("d"):
            firstFrame = None
        lastframe = frame

        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    video_capture.release()
    cv2.destroyAllWindows()

def distance(ptsA,ptsB,eps):
    if len(ptsA)==0:
        return True
    else:
        return np.sqrt((ptsA[0]-ptsB[0])**2+(ptsA[1]-ptsB[1])**2)>eps

#Human detection

In [9]:
# NOT WORKING YET
def human_detector(filename=""):
        # initialize the HOG descriptor/person detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    #cv2.startWindowThread()

    if len(filename)==0:
      video_capture = cv2.VideoCapture(2)
    else :
      video_capture = cv2.VideoCapture(filename)

    # the output will be written to output.avi
    
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()

        # resizing for faster detection
        frame = cv2.resize(frame, (640, 480))
        cv2.normalize(frame, frame, -100, 155, cv2.NORM_MINMAX)
        # using a greyscale picture, also for faster detection
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        # detect people in the image
        # returns the bounding boxes for the detected objects
        boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )

        boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

        for (xA, yA, xB, yB) in boxes:
            # display the detected boxes in the colour picture
            cv2.rectangle(frame, (xA, yA), (xB, yB),(0, 255, 0), 2)
        
        # Display the resulting frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    # finally, close the window
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Command Example 

In [ ]:
#human_detector()
#motion_sensor_v2()
#motion_tracker_multiple(2)
#mouth_open_close()
face_blur("")   #  enter filename here